# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2028ca1d00>
├── 'a' --> tensor([[-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679]])
└── 'x' --> <FastTreeValue 0x7f2028ca1cd0>
    └── 'c' --> tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                        [-0.0943, -1.8800,  0.1354,  1.2302],
                        [-0.1594, -0.8348,  1.9182,  0.1561]])

In [4]:
t.a

tensor([[-0.4181,  1.0037, -0.7925],
        [-0.4357, -1.8849,  0.2679]])

In [5]:
%timeit t.a

60.6 ns ± 0.0916 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2028ca1d00>
├── 'a' --> tensor([[ 0.8995, -1.4882,  1.9752],
│                   [-2.8196, -0.5224,  0.0594]])
└── 'x' --> <FastTreeValue 0x7f2028ca1cd0>
    └── 'c' --> tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                        [-0.0943, -1.8800,  0.1354,  1.2302],
                        [-0.1594, -0.8348,  1.9182,  0.1561]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0471 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4181,  1.0037, -0.7925],
               [-0.4357, -1.8849,  0.2679]]),
    x: Batch(
           c: tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561]]),
       ),
)

In [10]:
b.a

tensor([[-0.4181,  1.0037, -0.7925],
        [-0.4357, -1.8849,  0.2679]])

In [11]:
%timeit b.a

57 ns ± 0.0192 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1295, -0.1932,  0.5391],
               [ 0.5156, -0.5164, -0.1044]]),
    x: Batch(
           c: tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.504 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

887 ns ± 1.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 36.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 830 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1f7c9b8820>
├── 'a' --> tensor([[[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]],
│           
│                   [[-0.4181,  1.0037, -0.7925],
│                    [-0.4357, -1.8849,  0.2679]]])
└── 'x' --> <FastTreeValue 0x7f1f7c9b86d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.3 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f200043a310>
├── 'a' --> tensor([[-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679],
│                   [-0.4181,  1.0037, -0.7925],
│                   [-0.4357, -1.8849,  0.2679]])
└── 'x' --> <FastTreeValue 0x7f2028d0ee20>
    └── 'c' --> tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                        [-0.0943, -1.8800,  0.1354,  1.2302],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 46.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.3293,  0.7397,  1.7050, -1.4298],
                       [-0.0943, -1.8800,  0.1354,  1.2302],
                       [-0.1594, -0.8348,  1.9182,  0.1561]],
              
                      [[ 1.3293,  0.7397,  1.7050, -1.4298],
                       [-0.0943, -1.8800,  0.1354,  1.2302],
                       [-0.1594, -0.8348,  1.9182,  0.1561]],
              
                      [[ 1.3293,  0.7397,  1.7050, -1.4298],
                       [-0.0943, -1.8800,  0.1354,  1.2302],
                       [-0.1594, -0.8348,  1.9182,  0.1561]],
              
                      [[ 1.3293,  0.7397,  1.7050, -1.4298],
                       [-0.0943, -1.8800,  0.1354,  1.2302],
                       [-0.1594, -0.8348,  1.9182,  0.1561]],
              
                      [[ 1.3293,  0.7397,  1.7050, -1.4298],
                       [-0.0943, -1.8800,  0.1354,  1.2302],
                       [-0.1594, -0.8348,  1.9182,  0.1561]],

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 61.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561],
                      [ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561],
                      [ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561],
                      [ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561],
                      [ 1.3293,  0.7397,  1.7050, -1.4298],
                      [-0.0943, -1.8800,  0.1354,  1.2302],
                      [-0.1594, -0.8348,  1.9182,  0.1561],
                      [ 1.3293,  0.7397,  1.7050, -1.4298],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
